# Project Report: Stage One

#### Marcello Adhikary

## Background

In 2019, The Drug Enforcement Administration published a database that tracked the release and sale of every pain pill sold in the United States from 2006 through 2012, amounting to approximately 380 million transactions. This database allows us to take an in depth look at the influx of pain pills that sent the United States spiraling into the opioid epidemic that has decimated communities all around the country. In the 7 year period from 2006 to 2012, there were nearly 100,000 deaths recorded as a result of the prescription opioid crisis, including those who transitioned from prescription opioid drugs to street drugs such as heroin in order to maintain their addiction (“Drilling”, 1). 
	


  The life-destroying addictions caused by these drugs are running rampant in our communities. Before the release of this database, not much was known about the volume of these pills being provided, or their sources. Our data from the DEA provides us with important information such as the location of the sale of the pills, the manufacturer, the amount of pills per transaction, and much more. All of this will allow us to see where the opioid problem is most prevalent, and what companies are supplying the pills to these locations. 


## Data Acquisition

The data was provided by The Washington Post who retrieved it from the DEA after a court order. The retrieval of the database was the result of a “year-long legal battle for access to the database, which the government and the drug industry had sought to keep secret” (“Drilling, 2019). 
The data made available to the public via The Washington Post allowed the user to select subsets of the full database by state, and even by county. I decided that I wanted to analyze the data from the full country in order to get a more complete picture of the effects that opioids had in the United States. The file came as a zipped (compressed)  .tsv (Tab Separated Value) file and was 3.5GB in size. When unzipped, the data was over 80GB in total. The size of the file and the amount of computing power required to analyze it made it not viable to analyze on my local machine. I decided to move my project to a cloud computing solution. I selected Google’s Cloud Platform (GCP) as it provided me with ample tools to be able to analyze my data, including a Python environment in the form of a Jupyter notebook, with which I am very familiar and comfortable. GCP also provided me a $300 free trial credit, as the creation of  a Virtual Machine (VM) and using their cloud computing resources costs money.


## Current Progress

Opening the data has thusfar proven to be an immense struggle. I have attempted several different methods both in a Jupyter notebook, and on other Google Cloud Platfrom (GCP) utilities such as Bigtable. Below is some code that is not working to import the data to my notebook. 

In [ ]:
import pandas as pd
from io import BytesIO, StringIO
from google.cloud import storage
from google.oauth2 import service_account

def get_byte_fileobj(project: str,
                     bucket: str,
                     path: str,
                     service_account_credentials_path: str = None) -> BytesIO:
    """
    Retrieve data from a given blob on Google Storage and pass it as a file object.
    :param path: path within the bucket
    :param project: name of the project
    :param bucket_name: name of the bucket
    :param service_account_credentials_path: path to credentials.
           TIP: can be stored as env variable, e.g. os.getenv('GOOGLE_APPLICATION_CREDENTIALS_DSPLATFORM')
    :return: file object (BytesIO)
    """
    blob = _get_blob(bucket, path, project, service_account_credentials_path)
    byte_stream = BytesIO()
    blob.download_to_file(byte_stream)
    byte_stream.seek(0)
    return byte_stream

def get_bytestring(project: str,
                   bucket: str,
                   path: str,
                   service_account_credentials_path: str = None) -> bytes:
    """
    Retrieve data from a given blob on Google Storage and pass it as a byte-string.
    :param path: path within the bucket
    :param project: name of the project
    :param bucket_name: name of the bucket
    :param service_account_credentials_path: path to credentials.
           TIP: can be stored as env variable, e.g. os.getenv('GOOGLE_APPLICATION_CREDENTIALS_DSPLATFORM')
    :return: byte-string (needs to be decoded)
    """
    blob = _get_blob(bucket, path, project, service_account_credentials_path)
    s = blob.download_as_string()
    return s


def _get_blob(bucket_name, path, project, service_account_credentials_path):
    credentials = service_account.Credentials.from_service_account_file(
        service_account_credentials_path) if service_account_credentials_path else None
    storage_client = storage.Client(project=project, credentials=credentials)
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(path)
    return blob

The above code was sourced from a GCP online forum. It is meant to pull my data from my Bucket, but it did not work as intended from me. I spent some time trying to debug, but I was unable to find the source of the problem. 

In [ ]:
from google.cloud import storage

from io import BytesIO

client = storage.Client()

bucket = "marcello-capstone"

blob = storage.blob.Blob("mydata.tsv.gz",bucket)

content = blob.download_as_string()

train = pd.read_csv(BytesIO(content))

train.to_csv() if you like


This code was sourced from a Kaggle forum explaining how to read in a csv from a GCP Bucket. It worked with test data that I tried, but it was unable to read in a zipped file, which mine was. I then transitioned to try and unzip the data in Google Cloud Shell (like command prompt for my virtual machine).

When attempting to unzip my data in the cloud shell, it produced an error. The storage space allotted to my VM was not enough to unzip the over 80GB data file.

Here is the tutorial I followed for this: https://googlegenomics.readthedocs.io/en/latest/use_cases/compress_or_decompress_many_files/

In [ ]:
!gsutil cat gs://marcello-capstone/notebooks/jupyter/mydata.tsv.gz 
    | zcat | gsutil cp - gs://marcello-capstone/notebooks/jupyter/mydata.tsv.gz

This piece of code uses Google's gsutil (python utility that allows you to use Google's cloud shell commands in Python) would run for a few moments, and then return an error. I was unable to debug, but I suspect this did not work for the same reason that I was unable to unzip in my cloud shell. Inefficient resources on my machine. 

In [ ]:
import pandas as pd

 #Path to the object in Google Cloud Storage that you want to copy
sample_gcs_object = 'gs://marcello-capstone/pain-pills-in-the-usa.zip'

# Copy the file from Google Cloud Storage to Datalab
!gsutil cp $sample_gcs_object 'mydata.tsv.gz'

# Unzip the file
!unzip 'mydata.tsv.gz' 

This is another piece of  code that I found on the GCP forum. I was unable to debug this code as well at the time. I now know this is because my Jupyter notebook's version of pandas was outdated. 

In [ ]:
import google.datalab.storage as storage
import pandas as pd
from io import BytesIO
mybucket = storage.Bucket('marcello-capstone')
mydata = mybucket.object('mydata.tsv.gz')

This piece of code also did not work for me either, and I was unable to debug.

I then created a Bigtable instance in order to analyze my data that way. Bigtable is designed to process large amounts of data, so I thought it would be a great solution. I followed an online tutorial (below) in order to create a table and import a csv into it. That worked for me, so I tried to do the same with my data. I first downloaded a subset of the data so that I could get the header names of the table, and ran the code with my data. The code ran for several minutes (close to an hour) and then read back a 'BUILD SUCCESS' message. When I tried to read my new table, it came back blank. This apparently is a common issue with zipped tsv files on Google's Cloud Platform. It cannot read in zipped tsv's and will report a 'BUILD SUCCESS' regardless. 


https://cloud.google.com/community/tutorials/cbt-import-csv

## Future

Future attempts would feature me creating another VM instance in order to upgrade my specifications so that I will have more processing power. This may help with some of the above methods that have previously failed. Also, increasing the power in my Dataproc cluster may result in me being able to read in the data.